<a href="https://colab.research.google.com/github/OlegPozovnoy/MailRuMLHomeworks/blob/master/HW3/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [121]:
from google.colab import drive
drive.mount('/datasets/')

Drive already mounted at /datasets/; to attempt to forcibly remount, call drive.mount("/datasets/", force_remount=True).


In [122]:
!ls "/datasets/My Drive/datasets/"

players.pkl  results.pkl  tournaments.pkl


In [0]:
import pickle
import pandas as pd

In [0]:
tournaments = pd.read_pickle("/datasets/My Drive/datasets/tournaments.pkl")
#tournaments = pd.read_pickle("./tournaments.pkl")

tournaments_cleaned = [(v['id'], v['name'], v['dateStart'][0:4], v['type']['name'], v['questionQty']) for k,v in tournaments.items() if v['questionQty'] != None]
del tournaments

Создадим трейн и тест набор турниров.

In [125]:
train_id = [ v[0] for v in tournaments_cleaned if int(v[2])== 2019]
test_id = [v[0] for v in tournaments_cleaned if int(v[2])== 2020]

print(len(train_id))
print(len(test_id))

687
418


In [0]:
results = pd.read_pickle("/datasets/My Drive/datasets/results.pkl")
#results = pd.read_pickle("./results.pkl")
results_cleaned = {k: v for k,v in results.items() if k in train_id or k in test_id}

results_cleaned2 = {}
for k,v in results_cleaned.items():
  results_cleaned2[k] = []
  for s in v:
    if s.get('mask') != None:
      results_cleaned2[k].append((s.get('mask'),s.get('questionsTotal'), [k.get('player').get('id') for k in s.get('teamMembers')]))

del results
del results_cleaned

In [127]:
train_id = [id for id in train_id if id in results_cleaned2.keys() and len(results_cleaned2[id]) > 0]
test_id = [id for id in test_id if id in results_cleaned2.keys() and len(results_cleaned2[id]) > 0]

print(len(train_id))
print(len(test_id))

675
173


In [128]:
import numpy as np
from sklearn.linear_model import LogisticRegression
import tqdm
import gc

# creating all questions and players set
all_words = []
for k,v in results_cleaned2.items():
  for team in v: 
    for p in range(len(team[0])):
        all_words.append(str(k)+'_'+str(p))
    for m in team[2]:
        all_words.append(m)

all_words = set(all_words)
print(len(all_words))

# creating storage for competetion results
all_results = {}
for s in all_words:
  all_results[s] = [0,0] # total rating and num of competetions


logistic = LogisticRegression(fit_intercept = False)

def process_tournament(id):
  task3 = []
  words = []
   
  for n in range(len(results_cleaned2[id])): # command id
    for p in range(len(results_cleaned2[id][n][0])): #answers
        quest = str(id)+'_'+str(p)
        team = n
        task3.append((quest, team,results_cleaned2[id][n][0][p]) )
        words.append(quest)
        words.append(team)

  if len(words)==0:
    return 
  
  words = set(words) # encoding questions and teams no

  idx_to_word = dict(enumerate(words))
  word_to_idx = dict([(y,x) for x,y in idx_to_word.items()])
  #preparing matrix
  X = np.zeros((len(task3),len(words)))
  Y = np.zeros(len(task3))
  for i in range(len(task3)):
    X[i,word_to_idx[task3[i][0]]] = 1
    X[i,word_to_idx[task3[i][1]]] = 1
    Y[i] = 1 if str(task3[i][2]) == '1' else 0

  if len(set(Y)) <= 1:
    print ('<2 categories , skipping')
    return

  #print(X.shape)
  logistic.fit(X, Y)

  for s in range(len(logistic.coef_[0])):
    if isinstance(idx_to_word[s], int): ## teanm no
        #print('team '+ str(idx_to_word[s]))
        for player in results_cleaned2[id][idx_to_word[s]][2]:
            #print(player)
            all_results[player][1] +=1
            all_results[player][0] +=logistic.coef_[0][s]
    else: #question no
        #print('question '+ str(idx_to_word[s]))
        all_results[idx_to_word[s]][1] +=1
        all_results[idx_to_word[s]][0] +=logistic.coef_[0][s]

#process_tournament(1526)


104707


Пройдемся по всем турнирам и посчитаем рейтинг всех игроков и вопросов в заданном турнире, за неимением лучшего, усредним по каждому игроку рейтинг в турнире

In [129]:
import tqdm

for tourn_id in tqdm.tqdm(train_id):
  process_tournament(tourn_id)






  0%|          | 0/675 [00:00<?, ?it/s]




  0%|          | 1/675 [00:00<01:55,  5.84it/s]




  0%|          | 2/675 [00:02<09:54,  1.13it/s]




  0%|          | 3/675 [00:04<14:06,  1.26s/it]




  1%|          | 4/675 [00:07<18:01,  1.61s/it]




  1%|          | 5/675 [00:09<19:07,  1.71s/it]




  1%|          | 7/675 [00:09<14:06,  1.27s/it]




  1%|          | 8/675 [00:09<10:26,  1.07it/s]




  1%|▏         | 9/675 [00:10<08:19,  1.33it/s]




  1%|▏         | 10/675 [00:10<06:43,  1.65it/s]




  2%|▏         | 11/675 [00:10<05:31,  2.01it/s]




  2%|▏         | 12/675 [00:10<04:17,  2.58it/s]




  2%|▏         | 13/675 [00:11<06:30,  1.69it/s]




  2%|▏         | 14/675 [00:15<16:29,  1.50s/it]




  2%|▏         | 16/675 [00:15<11:51,  1.08s/it]




  3%|▎         | 17/675 [00:15<08:38,  1.27it/s]




  3%|▎         | 19/675 [00:15<06:15,  1.75it/s]




  3%|▎         | 22/675 [00:16<04:29,  2.42it/s]




  4%|▎         | 24/675 [00:17<05:03,  2.15it/s]




  4%|

<2 categories , skipping







100%|█████████▉| 673/675 [02:08<00:00,  3.68it/s]




100%|█████████▉| 673/675 [02:26<00:00,  3.68it/s]




100%|██████████| 675/675 [02:52<00:00,  3.90it/s]


In [160]:
all_ranking = {}

for k,v in all_results.items():
  #print(k,v)
  if v[1] != 0:
    all_ranking[k] = v[0]/v[1] 


print(all_results[30152])
print(all_ranking[30152])

[221.70377848329815, 130]
1.705413680640755


In [161]:
import statistics as stat

def is_player(t):
  return isinstance(t, int)
  
blank_rank = stat.mean ([ v for k,v in all_ranking.items() if is_player(k)])
print(blank_rank)

def get_team_ranking(players):
  result = []
  print('Team:',players)
  if (len(players)==0):
    return blank_rank
  for player in players:
    if (all_results[player][1]>0): # num of competetions
      print(player, ' rating is ', all_ranking[player])
      result.append(all_ranking[player])
    else:
      result.append(blank_rank)
  return stat.mean(result)


-0.5379797641338633


In [162]:
spearman = []
pearson = []

test_id = [x for x in test_id if len(results_cleaned2[x])>1] ## фильтруем турниры с 1 командой

for tourn_id in test_id:
  print('Tournament:', tourn_id)
  rating = []
  ranking = []
  i=1
  for team in results_cleaned2[tourn_id]:
    rating.append(get_team_ranking(team[2]))
    ranking.append(i)
    i+=1
  print ('Spearman: ',-stats.spearmanr(rating, ranking)[0])
  print ('Pearson: ',-stats.pearsonr(rating,ranking)[0])
  # ставим минус т.к. у нас места возрастают а сила убывает
  spearman.append(-stats.spearmanr(rating, ranking)[0])
  pearson.append(-stats.pearsonr(rating,ranking)[0])



Streaming output truncated to the last 5000 lines.
69936  rating is  0.789284765935024
88616  rating is  0.7518774273176628
91358  rating is  0.752674322736877
115555  rating is  0.6362151006612768
Team: [10695, 19915, 23178, 26911, 74382]
10695  rating is  1.3207852682856311
19915  rating is  1.423074699045017
23178  rating is  1.3967236686545361
26911  rating is  1.2992712132278228
74382  rating is  1.3949971489615483
Team: [39185, 90800, 101725, 104901, 104902, 113694]
39185  rating is  0.32783276988709803
90800  rating is  0.3864237460536274
101725  rating is  0.3302095620604455
104901  rating is  0.37796698481449237
104902  rating is  0.27307212097008754
113694  rating is  0.3607437423728309
Team: [2115, 67807, 111656, 116051, 191655]
2115  rating is  0.5973821865289993
67807  rating is  0.6266829097481287
111656  rating is  0.6351281376016952
116051  rating is  0.0030552551231076506
191655  rating is  -0.06729502980606195
Team: [19158, 39366, 51852, 132228, 134429, 137380, 176349

In [163]:
print(spearman)
print(pearson)
print(stat.mean(spearman))
print(stat.mean(pearson))

[0.7302837415039378, 0.7769638794729264, 0.7857266542980827, 0.7571479145821264, 0.5878069812176181, 0.7219305452239768, 0.6338391968235293, 0.7020631281960853, 0.5958556798082136, 0.4168306855170206, 0.5182010463585892, 0.8440030750950487, 0.6928426044300088, 0.6432143638468918, 0.6397192764003768, 0.5945538602214889, 0.8308823529411766, 0.7972045094498877, 0.8032883425753955, 0.7004561824729891, 0.821067714289558, 0.8381972265023113, 0.7826655282623971, 0.8164776477647764, 0.6346326739079873, 0.7690484126021944, 0.755033462365203, 0.617391304347826, 0.836171139896661, 0.787268817204301, 0.8869021782814887, 0.8590717678970162, 0.7640917469056793, 0.7668308702791461, 0.882706766917293, 0.8022405775523899, 0.748051948051948, 0.7898210568662624, 0.7693191013030302, 0.7615320790320791, 0.812891986062718, 0.7277026368207713, 0.7622377622377624, 0.9009304871373838, 0.7033580067453409, 0.8251082251082253, 0.8128977114315507, 0.8183650615901457, 0.7867509848641924, 0.8587219541924115, 0.81810

Теперь будем работать в рамках модели, когда участники команды отвечают на вопрос независимо и если кто то ответил - команда отвечает правильно, если никто - не отвечает. Будем попеременно улучшать оценки для сложностей вопросов и индивидуальных показателей всех участников

In [0]:
from scipy.special import expit
import math

def sigmoid(x):
  return expit(x)

def sigm_reverse(x):
  return -math.log(1/x-1)

def adjust_individual(players, team_X, Y):
  prob_real = 1-sigmoid(team_X + Y)
  prob_adjusted = 1
  probs = []
  for player in players:
    player_rating = all_ranking[player]
    player_prob = (1 - sigmoid(player_rating + Y))
    probs.append(player_prob)
    prob_adjusted *= (1 - sigmoid(player_rating + Y))
  multiplier = math.pow(prob_real/prob_adjusted, 1.0/len(players))
  probs = [x*multiplier for x in probs]
  result = [sigm_reverse(1-x) - Y for x in probs]

def compose_team(players, team_X, Y):
  prob_adjusted = 1
  for player in players:
    player_rating = all_ranking[player]
    prob_adjusted *= (1 - sigmoid(player_rating + Y))
  return sigm_reverse(1-prob_adjusted) - Y



In [0]:
print(results_cleaned2[])

In [0]:
# creating storage for competetion results
all_results_upd = {}
for s in all_words:
  all_results_upd[s] = [0,0] # total rating and num of competetions


logistic = LogisticRegression(fit_intercept = False)

def upd_questions(id):
  task3 = []
  words = []
   
  for n in range(len(results_cleaned2[id])): # command id
    for p in range(len(results_cleaned2[id][n][0])): #answers
        quest = str(id)+'_'+str(p)
        team = n
        task3.append((quest, team,results_cleaned2[id][n][0][p]))
        words.append(quest)
        words.append(team)

  if len(words)==0:
    return 
  
  words = set(words) # encoding questions and teams no

  idx_to_word = dict(enumerate(words))
  word_to_idx = dict([(y,x) for x,y in idx_to_word.items()])
  #preparing matrix
  X = np.zeros((len(task3),len(words)))
  Y = np.zeros(len(task3))
  for i in range(len(task3)):
    X[i,word_to_idx[task3[i][0]]] = 1
    X[i,word_to_idx[task3[i][1]]] = 1
    Y[i] = 1 if str(task3[i][2]) == '1' else 0

  if len(set(Y)) <= 1:
    print ('<2 categories , skipping')
    return

  #print(X.shape)
  logistic.fit(X, Y)

  for i in range(len(task3)):
    X[i,word_to_idx[task3[i][0]]] = 1
    X[i,word_to_idx[task3[i][1]]] = 1
    Y[i] = 1 if str(task3[i][2]) == '1' else 0


  for i in range(len(task3)):
    question = word_to_idx[task3[i][0]]
    team = word_to_idx[task3[i][0]]
    k_quest = logistic.coef_[0][question]
    k_team = logistic.coef_[0][team]
    K = compose_team(results_cleaned2[id][idx_to_word[i]][2], k_team,all_ranking[task3[i][0]] )
    print(task3[i][0], k_quest,k_team,  K)
    all_results_upd[task3[i][0]][1] += 1
    all_results_upd[task3[i][0]][0] += k_quest + k_team - K


  #for s in range(len(logistic.coef_[0])):
  #  if isinstance(idx_to_word[s], int): ## teanm no
  #      #print('team '+ str(idx_to_word[s]))
  #      for player in results_cleaned2[id][idx_to_word[s]][2]:
  #          #print(player)
  #          all_results_upd[player][1] +=1
  #          all_results_upd[player][0] +=logistic.coef_[0][s]
  #  else: #question no
  #      #print('question '+ str(idx_to_word[s]))
  #      all_results_upd[idx_to_word[s]][1] +=1
  #      all_results_upd[idx_to_word[s]][0] +=logistic.coef_[0][s]



In [170]:
import tqdm

def upd_all_questions():
  for tourn_id in tqdm.tqdm(train_id):
    upd_questions(tourn_id)


upd_all_questions()






  0%|          | 0/675 [00:00<?, ?it/s]


KeyError: ignored